<a href="https://colab.research.google.com/github/sophianicogos/VigenereCOLOR/blob/main/toolboxHCP_retinotopy_averageSUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ccnmaastricht/CNI_toolbox.git


Cloning into 'CNI_toolbox'...
remote: Enumerating objects: 1411, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 1411 (delta 150), reused 267 (delta 141), pack-reused 1118
Receiving objects: 100% (1411/1411), 943.45 KiB | 10.25 MiB/s, done.
Resolving deltas: 100% (710/710), done.


### ♦ PACKAGE SETUP ♦

In [2]:
import nibabel as nib
import numpy as np

In [51]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_file_path = '/cni_tlbx.zip'  # Adjust the path if necessary

# Destination directory for extraction
extraction_path = '/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the contents of the extraction directory
os.listdir(extraction_path)


import os
os.chdir('/')
!pwd
from cni_tlbx import IRM


/


### DATA HANDLING

In [4]:
NIFTI = nib.load('/content/GlasserParcellation_niftiConvert.nii')
header = NIFTI.header
affine = NIFTI.affine
data = np.array(NIFTI.dataobj)
data = np.transpose(data, (3, 0, 1, 2))

### pRF MAPPING

instantiation?

Input referred models?

In [52]:
from IRM import IRM
data = data  # Assuming you have your functional MRI data tensor
sampling_frequency = 1  # Sampling frequency of data acquisition

# Extracting dimensions of the data tensor
n_samples, n_rows, n_cols, n_slices = data.shape

# Creating a dictionary of parameters
parameters = {
    'f_sampling': sampling_frequency,
    'n_samples': n_samples,
    'n_rows': n_rows,
    'n_cols': n_cols,
    'n_slices': n_slices
}

# Instantiating the IRM tool with the parameters
irm = IRM(parameters)


usage..

In [ ]:
stimulus_file_path = ""
irm.set_stimulus(stimulus_file_path)

irm.create_timecourse(f,xdata)
results = irm.mapping(data)

### Saving resuts (future)

In [ ]:
header["dim"][0] = 3
header["dim"][4] = 1

for key, value in results.items():
    results_nii = nib.Nifti1Image(value, affine, header)
    nib.save(results_nii, "%s.nii.gz" % key)


can i combine these?